In [2]:
import pandas as pd
import numpy as np
import os
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import json
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
stopWords = stopwords.words("english")
stopWords.append("i\'ve")
stopWords.append("i\'m")
stopWords.append("he\'d")
stopWords.append("she\'d")
stopWords.append("i\'d")
stopWords.append("i\'ll")
stopWords.append("we\'ll")

In [5]:
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word in sentence.split():
            lemmatized_sentence.append(lemmatizer.lemmatize(word))
    return " ".join(lemmatized_sentence)

In [6]:
def special_characters_removal(word):
  special=list(string.punctuation)
  special.append(' ')
  special.remove('*')
  x=[]
  for i in word:
    if i not in special:
      x.append(i)
    else:
      x.append(' ')
  return ''.join(x)  

In [7]:
def preprocessing(snippet):
  snippet=snippet.split()
  res=[]
  for i in range(len(snippet)):
    if snippet[i] not in stopWords and len(snippet[i])>1:
      res.extend(special_characters_removal(snippet[i]).split())
  snippet=" ".join(res)
  snippet=lemmatize_sentence(snippet)
  return snippet.lower()  

In [8]:
def inverted_index_construction(file,location,posting_list):
  file_df=pd.read_csv(location+file)
  snippets=list(file_df['Snippet'])
  result=[]
  for i in snippets:
    result.append(preprocessing(i))
  for i in range(len(result)):
    tokens=result[i].split()
    for k in range(len(tokens)):
      j=tokens[k]
      if j in posting_list:
        if file in posting_list[j]:
          posting_list[j][file]['frequency']+=1
          if i in posting_list[j][file]['records']:
            posting_list[j][file]['records'][i]['frequency']+=1
            posting_list[j][file]['records'][i]['position'].append(k)
          else:
            posting_list[j][file]['records'][i]={'position':[k],'frequency':1}
        else:
          posting_list[j]['doc_frequency']+=1
          posting_list[j][file]={'frequency':1,'records':{i:{'position':[k],'frequency':1}}}
      else:
        posting_list[j]={file:{'frequency':1,'records':{i:{'position':[k],'frequency':1}}},'doc_frequency':1}

In [9]:
posting_list={}
files=os.listdir('drive/My Drive/archive/TelevisionNews')
location='drive/My Drive/archive/TelevisionNews/'
for i in range(len(files)):
  try:
    inverted_index_construction(files[i],location,posting_list)
  except:
    print(files[i])
f=open('drive/My Drive/AIR/inverted_index1.json','w')
f.write(json.dumps(posting_list))
f.close()
%time

CNN.200910.csv
CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.54 µs


In [10]:
def permutem(word,permutem_index):
  i=0
  while(i<=len(word)):
    x=word[i:]+"$"+word[:i]
    permutem_index[x]=word
    i+=1

In [11]:
permutem_index={}
for i in list(posting_list.keys()):
  permutem(i,permutem_index)

In [12]:
f=open('drive/My Drive/AIR/permuterm1.json','w')
f.write(json.dumps(permutem_index))
f.close()

In [13]:
files=os.listdir('drive/My Drive/archive/TelevisionNews')
files.sort()

In [ ]:
import pandas as pd
record_list={}
location='drive/My Drive/archive/TelevisionNews/'
prev=0
for i in files:
  try:
    df=pd.read_csv(location+i)
    record_list[i]=[len(df),prev]
    prev+=len(df)
  except:
    print(i)
    pass
f=open('drive/My Drive/AIR/record_list.json','w')
f.write(json.dumps(record_list))
f.close()

CNN.200910.csv


In [14]:
len(posting_list)

38231

In [15]:
import pandas as pd
final=[]
location='drive/My Drive/archive/TelevisionNews/'
for i in files:
  try:
    df=pd.read_csv(location+i)
    records=df.values.tolist()
    final.extend(list(map(lambda x:[records[x][-1],i,x],range(len(records)))))
  except:
    pass
f=open('drive/My Drive/AIR/snippet_df.txt','w')
f.write(str(final))
f.close()

In [16]:
final[:5]

[['beena part to do. the airline industry has not been a part of this move to reduce carbon and teal last year. -- and teal. they agreed on a deal to curb greenhouse gases sol',
  'BBCNEWS.201701.csv',
  0],
 ["it's beaten it by about 0.1, 0.12 degrees celsius. which doesn't seem like a lot, but in terms of the yearly variations, it is actually huge. part of this rise was caused by an el nino event, a warm ocean current that disrupts the world's weather. but scientists say greenhouse gases",
  'BBCNEWS.201701.csv',
  1],
 ['contact more than expected, how. your co nta ct le ns more than expected, how. your contact lens has come out of your eye, good catch!. what does i do in terms of policy, already there is a pushback on global warming. interesting to see this against the backdrop of president trump coming into the oval office, saying things like climate change is something',
  'BBCNEWS.201701.csv',
  2],
 ['where every time a marketplace is closed down, another appears to replace it?

In [17]:
preprocessing("president trump OR climate change")

'president trump or climate change'